<a href="https://colab.research.google.com/github/2303A52158/GENERATIVE-AI_2025/blob/main/Generative_AI_2303A52158_Ass_8_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**1. (1 ponto) Design a multilayer ANN architecture according to the requirements shown below.
Train, test, save (.h5) and deploy the model to predict the quality of wine using the Keras deep
learning library**

**2. (1 ponto) Calculate training and testing accuracy, build confusion matrix, also calculate recall,
precision and F1-score.**

**3. (1 ponto) Build the application by loading the saved ANN model.**

                              Tabela 1: ANN Architecture
              Layer         Neurons Activation Function
          Hidden Layer - 1    16         sigmoid
          Hidden Layer - 2    24         sigmoid
          Hidden Layer - 3    10         sigmoid

                              Tabela 2: Training Parameters

                        epochs batch-size error-metric Optimizer
                        260        64       accuracy    rmsprop

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

df = pd.read_csv("/content/winequality-red.csv")

X = df.drop(columns=['quality']).values
y = df['quality'].values

encoder = LabelEncoder()
y_encoded = encoder.fit_transform(y)
y_categorical = to_categorical(y_encoded)

X_train, X_test, y_train, y_test = train_test_split(X, y_categorical, test_size=0.2, random_state=42, stratify=y_encoded)


scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

model = Sequential([
    Dense(16, activation='sigmoid', input_shape=(X_train.shape[1],)),
    Dense(24, activation='sigmoid'),
    Dense(10, activation='sigmoid'),
    Dense(y_categorical.shape[1], activation='softmax')
])

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=260, batch_size=64, validation_data=(X_test, y_test))

y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)

accuracy = accuracy_score(y_true, y_pred_classes)
conf_matrix = confusion_matrix(y_true, y_pred_classes)
report = classification_report(y_true, y_pred_classes)

print(f"Test Accuracy: {accuracy:.4f}")
print("Confusion Matrix:")
print(conf_matrix)
print("Classification Report:")
print(report)

model.save("wine_quality_model.h5")

def predict_wine_quality(features):
    model = tf.keras.models.load_model("wine_quality_model.h5")
    features = scaler.transform([features])
    prediction = model.predict(features)
    return encoder.inverse_transform([np.argmax(prediction)])

sample_features = X_test[0]
predicted_quality = predict_wine_quality(sample_features)
print(f"Predicted Wine Quality: {predicted_quality[0]}")


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/260
20/20 ━━━━━━━━━━━━━━━━━━━━ 8s 112ms/step - accuracy: 0.0279 - loss: 2.3664 - val_accuracy: 0.0344 - val_loss: 2.0997
Epoch 2/260
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - accuracy: 0.0375 - loss: 2.0310 - val_accuracy: 0.0344 - val_loss: 1.8839
Epoch 3/260
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - accuracy: 0.1468 - loss: 1.8540 - val_accuracy: 0.4250 - val_loss: 1.7231
Epoch 4/260
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.4184 - loss: 1.7010 - val_accuracy: 0.4250 - val_loss: 1.5983
Epoch 5/260
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.4126 - loss: 1.5838 - val_accuracy: 0.4250 - val_loss: 1.5025
Epoch 6/260
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - accuracy: 0.3974 - loss: 1.5043 - val_accuracy: 0.4250 - val_loss: 1.4271
Epoch 7/260
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - accuracy: 0.4402 - loss: 1.3928 - val_accuracy: 0.4250 - val_loss: 1.3669
Epoch 8/260
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.4429 - loss: 1.3515 - val_accuracy: 0

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Test Accuracy: 0.6062
Confusion Matrix:
[[ 0  0  1  1  0  0]
 [ 0  0 10  1  0  0]
 [ 0  0 98 38  0  0]
 [ 0  0 42 80  6  0]
 [ 0  0  1 23 16  0]
 [ 0  0  0  2  1  0]]
Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         2
           1       0.00      0.00      0.00        11
           2       0.64      0.72      0.68       136
           3       0.55      0.62      0.59       128
           4       0.70      0.40      0.51        40
           5       0.00      0.00      0.00         3

    accuracy                           0.61       320
   macro avg       0.32      0.29      0.30       320
weighted avg       0.58      0.61      0.59       320



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
Predicted Wine Quality: 6
